# 1  Introduction

Team: 
<par> David Grogan (SCPD Student) 
<par> Peter Gjeltema (SCPD Student)

# 2 Choosing a dataset 

<par> Project Topic: MLB Baseball - Pitching Data
<par> Dataset: Lahman's Baseball Database (https://cran.r-project.org/web/packages/Lahman/Lahman.pdf)

# 3 Register a Team

[X] COMPLETE

# 4 Setting Aside a Holdout

In [1]:
# load packages
library(Lahman)
library(tidyverse)
library(GGally)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --
v ggplot2 3.0.0     v purrr   0.2.5
v tibble  1.4.2     v dplyr   0.7.6
v tidyr   0.8.1     v stringr 1.3.1
v readr   1.1.1     v forcats 0.3.0
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Attaching package: 'GGally'

The following object is masked from 'package:dplyr':

    nasa



In [2]:
?Lahman

In [3]:
# create data set combining playoff and reg season pitching stats
cat("Pitching table dim: ", dim(PitchingPost), "\n")
aug_pitch = inner_join(PitchingPost, Pitching, by=c("playerID", "yearID")) %>% na.omit()
cat("Pitcher w/ Postseason clean dim: ", dim(aug_pitch), "\n")

##PROBLEM - DOES CLEANED DATASET GIVE ENOUGH VALUES?

Pitcher_clean = Pitching %>% na.omit()
#Pitcher_clean = Pitching[complete.cases(Pitching), ]
cat("Pitcher_clean dim: ", dim(Pitcher_clean))

## THIS SEEMS MORE REASONABLE - LARGER N, SMALLER P

Pitching table dim:  5271 30 
Pitcher w/ Postseason clean dim:  2744 58 
Pitcher_clean dim:  12047 30

In [8]:
set.seed(1)
set_size = dim(Pitcher_clean)[1]
train = sample(seq(set_size), 0.8*set_size)
clean_pitching_train = Pitcher_clean[train, ]
clean_pitching_test = Pitcher_clean[-train, ]

# 5 Investigating and Exploring the Data

In [4]:
# basic stats on dataset
head(Pitcher_clean)
summary(Pitcher_clean)
sapply(Pitcher_clean, sd, na.rm=TRUE)

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
32901,abbotpa01,2000,1,SEA,AL,9,7,35,27,0,...,4,3,5,0,766,2,89,1,4,20
32902,aceveju01,2000,1,MIL,NL,3,7,62,0,0,...,9,3,1,2,347,18,38,1,1,7
32903,adamste01,2000,1,LAN,NL,6,9,66,0,0,...,0,5,0,0,369,18,42,3,0,9
32904,aguilri01,2000,1,CHN,NL,1,2,54,0,0,...,2,1,4,0,210,44,28,1,0,3
32905,aldresc01,2000,1,PHI,NL,1,3,23,0,0,...,0,1,1,0,95,5,14,1,2,1
32906,alfonan01,2000,1,FLO,NL,5,6,68,0,0,...,3,0,1,2,311,62,35,3,1,10


   playerID             yearID         stint           teamID     lgID     
 Length:12047       Min.   :2000   Min.   :1.000   TEX    : 457   AA:   0  
 Class :character   1st Qu.:2004   1st Qu.:1.000   SDN    : 448   AL:5682  
 Mode  :character   Median :2008   Median :1.000   CLE    : 436   FL:   0  
                    Mean   :2008   Mean   :1.085   COL    : 436   NA:   0  
                    3rd Qu.:2013   3rd Qu.:1.000   NYA    : 428   NL:6365  
                    Max.   :2016   Max.   :4.000   TOR    : 421   PL:   0  
                                                   (Other):9421   UA:   0  
       W                L                G               GS        
 Min.   : 0.000   Min.   : 0.000   Min.   : 1.00   Min.   : 0.000  
 1st Qu.: 0.000   1st Qu.: 0.000   1st Qu.: 9.00   1st Qu.: 0.000  
 Median : 2.000   Median : 2.000   Median :22.00   Median : 0.000  
 Mean   : 3.427   Mean   : 3.427   Mean   :26.41   Mean   : 6.856  
 3rd Qu.: 5.000   3rd Qu.: 5.000   3rd Qu.:35.00   3

Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
"NAs introduced by coercion"Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
"Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector."Warning message in var(if (is.vector(x) || is.factor(x)) x else as.double(x), na.rm = na.rm):
"Calling var(x) on a factor x is deprecated and will become an error.
  Use something like 'all(duplicated(x)[-1L])' to test for a constant vector."

playerID      yearID       stint      teamID        lgID           W 
         NA   4.9367408   0.2918846  41.0243274   1.4976495   4.2754101 
          L           G          GS          CG         SHO          SV 
  3.7155361  21.1084848  10.8548390   0.7569531   0.3699451   6.6433928 
     IPouts           H          ER          HR          BB          SO 
181.3808642  60.4110962  27.7921430   7.4442195  20.2184436  49.0864279 
      BAOpp         ERA         IBB          WP         HBP          BK 
  0.0759872   5.8070264   2.0171929   2.6486385   2.8256999   0.5448314 
        BFP          GF           R          SH          SF        GIDP 
254.7506246  11.1156685  30.1105459   2.7109244   2.1217623   5.9909074

In [5]:
#Pitcher_clean$playerID = as.factor(Pitcher_clean$playerID)
#Pitcher_clean$yearID = as.factor(Pitcher_clean$yearID)
#Pitcher_clean$stint = as.factor(Pitcher_clean$stint)
#Pitcher_clean$teamID = as.factor(Pitcher_clean$teamID)
#Pitcher_clean$lgID = as.factor(Pitcher_clean$lgID)
#head(Pitcher_clean)
#cor(Pitcher_clean)

## CORRELATIONS

Pitcher_clean$lgID = ifelse(Pitcher_clean$lgID=="AL", 1, 0)
#head(Pitcher_clean)
cormat = round(cor(Pitcher_clean[, -c(1, 2, 3, 4)]), 2)
melted_cormat = melt(cormat)
# ggplot(data = melted_cormat, aes(x=Var1, y=Var2, fill=value)) + 
#   geom_tile()

# Get lower triangle of the correlation matrix
  get_lower_tri<-function(cormat){
    cormat[upper.tri(cormat)] <- NA
    return(cormat)
  }
  # Get upper triangle of the correlation matrix
  get_upper_tri <- function(cormat){
    cormat[lower.tri(cormat)]<- NA
    return(cormat)
  }

upper_tri <- get_upper_tri(cormat)
#upper_tri

# Melt the correlation matrix
melted_cormat <- melt(upper_tri, na.rm = TRUE)
# Heatmap
ggplot(data = melted_cormat, aes(Var2, Var1, fill = value))+
 geom_tile(color = "white")+
 scale_fill_gradient2(low = "blue", high = "red", mid = "white", 
   midpoint = 0, limit = c(-1,1), space = "Lab", 
   name="Pearson\nCorrelation") +
  theme_minimal()+ 
 theme(axis.text.x = element_text(angle = 45, vjust = 1, 
    size = 12, hjust = 1))+
 coord_fixed()

ERROR: Error in melt(cormat): could not find function "melt"


In [ ]:
# correlations specific to wins
ggpairs(Pitcher_clean[ , c(6, 9, 13, 18, 25)])

In [ ]:
# correlations specific to ER
ggpairs(Pitcher_clean[ , c(6, 9, 13, 18, 25)])